In [31]:
from urllib.request import urlopen
import json
import pandas as pd
import plotly.express as px
import plotly.io as pio
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)#This loads the counties.
    
data = pd.read_csv('https://raw.githubusercontent.com/nurfnick/COVID_Travel/master/Combined.csv',dtype={"FIPS": str})
#Combined.csv does not contain all counties.  Missing counties lacking google mobility data

#df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv",
 #                  dtype={"fips": str})#This loads the data




In [33]:
fig = px.choropleth(data,   geojson=counties, locations='FIPS', color='COVID Cases',
                           color_continuous_scale="Inferno",
                           animation_frame='Date',
                           animation_group='FIPS',
                           range_color =(0, 12000),
                           scope="usa",
                           #labels={'6/13/20':'Covid Cases on 6/13/20'},
                           hover_name = 'County'
                          )
#fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(template='plotly_white')
fig.update_layout(title='COVID cases by County in US')
pio.write_html(fig, file='coloropleth.html', auto_open=True)

In [41]:
pd.unique(data[data.State=='Alaska'].FIPS)

array(['02020', '02050', '02090', '02110', '02122', '02130', '02150',
       '02170', '02185', '02220', '02240', '02261'], dtype=object)

This will generate a cholorpleth with most counties represented.  Unfortunately sparcely populated counties are ignored because the google data did not use them.  This mistake persists in the first iteration of 'Combined.csv' so it cannot be fixed.  We are only using COVID cases on this map so we should be able to generate a full(er) map.

In [83]:
COVID = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')

In [118]:
COVID.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20
0,,AS,ASM,16,60.0,NaN,NaN,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,,GU,GUM,316,66.0,NaN,NaN,US,13.4443,144.7937,...,222,225,226,231,247,247,247,253,257,267
2,,MP,MNP,580,69.0,NaN,NaN,US,15.0979,145.6739,...,30,30,30,30,30,30,30,30,30,30
3,,PR,PRI,630,72.0,NaN,NaN,US,18.2208,-66.5901,...,6564,6685,6820,6877,6922,7066,7189,7250,7465,7537
4,,VI,VIR,850,78.0,NaN,NaN,US,18.3358,-64.8963,...,76,76,76,80,81,81,81,81,81,90


In [77]:
#pd.to_datetime(COVID.columns[11:])
dates = COVID.columns[64:]

In [100]:
COVID['UID']=COVID['UID'].astype('str')

In [106]:
COVID['UID'] = COVID.UID.str.slice(3,10)

In [114]:
COVID['Province_State'] = COVID.Admin2 +", " +COVID.Province_State

In [115]:
CovidSmoothed = pd.melt(COVID,id_vars=['UID','FIPS','Admin2','Province_State'],value_vars=dates)

In [116]:
fig = px.choropleth(CovidSmoothed,   geojson=counties, locations='UID', color='value',
                           color_continuous_scale="Inferno",
                           animation_frame='variable',
                           animation_group='UID',
                           range_color =(0, 12000),
                           scope="usa",
                           hover_name = 'Province_State',
                           labels={'value':'Covid Cases','Province_State':'County','variable':'Date'},
                          )
#fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(template='plotly_white')
fig.update_layout(title='COVID cases by County in US')
pio.write_html(fig, file='choropleth.html', auto_open=True)

I think we really need to get population in this computation and do infections as a percent of the population.